In [1]:
from guerrilla.device import Device
import time

In [2]:
config = {
    "name": "EDR-G9010",
    "type": "router",
    "host": "192.168.127.254",
    "username": "admin",
    "password": "moxa",
    "protocol": "ssh",
}
device = Device(config)
device.connect()

2023-11-07 18:44:48.293 | INFO     | guerrilla.connection.ssh:_establish_connection:36 - Created EDR-G9010's SSH session to 192.168.127.254:22
2023-11-07 18:44:48.629 | SUCCESS  | guerrilla.connection.ssh:_establish_connection:49 - Connected to admin 192.168.127.254:22
2023-11-07 18:44:52.386 | INFO     | guerrilla.connection.session:set_base_prompt:658 - Default Prompt:Firewall/VPN Router 00000
2023-11-07 18:44:52.397 | INFO     | guerrilla.device.router.router:disable_paging:76 - Disable Paging Default
2023-11-07 18:44:53.495 | INFO     | guerrilla.connection.session:set_base_prompt:658 - Default Prompt:Firewall/VPN Router 00000(config)


In [ ]:
device.session.original_prompt
device.session.base_prompt

In [ ]:
device.run('configure', expect_string=r"\(config.*\)#")
device.run('interface wan', expect_string=r"\(config.*\)#")

In [ ]:
device.session.write_channel('exit')

In [ ]:
device.find_prompt()

In [ ]:
current_prompt = device.find_prompt()[:-1]
while current_prompt != device.session.original_prompt:
    print("current_prompt", current_prompt)
    print("original_prompt", device.session.original_prompt)
    device.session.write_channel('exit')
    time.sleep(0.2)
    current_prompt = device.find_prompt()[:-1]

In [ ]:
device.run("hostname test456")

In [ ]:
device.show_system()

In [ ]:
device.session.base_prompt

In [ ]:
device.session.set_base_prompt()

In [ ]:
device.find_prompt()

In [ ]:
device.run('exit', expect_string=r".*(#|>)")

In [ ]:
device.check_config_mode()

In [ ]:
device.find_prompt()

In [ ]:
device.run('interface wan', expect_string=r"\(config.*\)#")

In [ ]:
device.config_mode('interface wan')

In [ ]:
device.run('interface wan', expect_string=r"\(config.*\)#")

In [ ]:
device.find_prompt()

In [ ]:
device.run_timing('reload factory-default')
device.run_timing('Y')
device.disconnect()
time.sleep(30)
device.connect()

In [ ]:
dut_iface_ip = '192.168.128.254'
dut_iface_vlan = '2'
dut_iface_port = '8'
dut_iface_mask = '255.255.255.0'

# device.config_mode('configure') # config
device.run_timing('configure') # config

device.run_timing(f'vlan create {dut_iface_vlan}')
# device.create_vlan(dut_iface_vlan)

device.run_timing(f'interface ethernet 1/{dut_iface_port}') # config-if


device.run_timing(f'switchport access vlan {dut_iface_vlan}') 
from guerrilla.logging import logger
import sys
logger.add(sys.stdout, level="DEBUG")
device.run_timing('exit') # return config
# device.exit_config_mode()
device.run_timing('interface wan') # config-if wan
device.run_timing(f'bind vlan {dut_iface_vlan}')
device.run_timing(f'ip address static {dut_iface_ip} {dut_iface_mask}')

# device.exit_config_mode() # return config
# device.exit_config_mode() # return main
device.run_timing('exit') # return config
device.run_timing('exit') # return main